<a href="https://colab.research.google.com/github/ESiler/umich_mads/blob/main/data_pipeline_tidied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Imports**

In [ ]:
#load libraries
import pandas as pd
import json
import datetime as dt
import numpy as np
from datetime import datetime, timedelta
import os as os
import pandas.api.types as ptypes
import re
import csv
import string
import openpyxl
import sys
import ast
from google.colab import drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# **1. Configure filepath names**

In [ ]:
## Ensure that you have entered the correct file paths for your system:
base_data_path = '/content/drive/My Drive/SIADS_593_Milestone_I/'
rfolder_path = base_data_path + 'nlrb_downloads/r_cases'
cfolder_path = base_data_path + 'nlrb_downloads/c_cases'
jsonfolder_path = base_data_path + 'nlrb_downloads/related_cases'
cleaned_data_path = base_data_path + 'cleaned_merged_data'

filepaths ={("base_data_path", base_data_path),
 ("rfolder_path", rfolder_path), ("cfolder_path", cfolder_path),
  ("jsonfolder_path", jsonfolder_path), ("cleaned_data_path", cleaned_data_path)}

for key, value in filepaths:
  if os.path.exists(value):
    print(key, "exists :)")
  else:
    print(key, "does not exist :( \n    Attempt to locate this file before proceeding and check your filename:", value)

cleaned_data_path exists :)
base_data_path exists :)
jsonfolder_path exists :)
rfolder_path exists :)
cfolder_path exists :)


# **3. Data Intake**

### **3a. Load R Case Dataset**

In [ ]:
def rmerge_csv_files(rfolder_path):
    dfs = []
    for filename in os.listdir(rfolder_path):
        if filename.endswith('.csv'):
            rfile_path = os.path.join(rfolder_path, filename)
            try:
                # Attempt to read the CSV file
                df = pd.read_csv(rfile_path)
                dfs.append(df)
            except pd.errors.ParserError as e:
                # If a ParserError occurs, print the file path and re-raise the error
                print(f"Error in file: {rfile_path}")
                raise e
    # Proceed to merge dataframes if no errors were raised
    rmerged_df = pd.concat(dfs, ignore_index=True)
    # Optionally, save the merged dataframe to a new CSV file
    rmerged_csv_path = os.path.join(rfolder_path, 'rmerged.csv')
    rmerged_df.to_csv(rmerged_csv_path, index=False)
    return rmerged_csv_path

In [ ]:
rmerged_csv_path = rmerge_csv_files(rfolder_path)
df = pd.read_csv(rmerged_csv_path, low_memory=False)
print("df length:", len(df))
df.sample(2)

df length: 60075


,Case Type,Region,Case Number,Case Name,Status,Date Filed,Date Closed,Reason Closed,City,States & Territories,Employees on charge/petition,Participants,Union,Unit Sought,Voters
4133,R,"Region 04, Philadelphia, Pennsylvania",04-RC-294422,"Aramark Educational Services, LLC at Kutztown ...",Closed,04/20/2022,07/11/2022,Certific. of Representative,Kutztown University,PA,100.0,"Employer, Legal Representative, Trachsel Grego...","Pennsylvania Joint Board, Workers United, SEIU",Included: All full time and regular part time...,97.0
40496,R,"Region 04, Philadelphia, Pennsylvania",04-RM-328041,"AC Ocean Walk, LLC d/b/a Ocean Casino Resort",Closed,10/17/2023,12/15/2023,Withdrawal Non-adjusted,Atlantic City,NJ,9.0,"Petitioner, Legal Representative, Nagle Robert...",NaN,Included: Local 54 seeks recognition for Traf...,NaN


### **3b. Load C Case Dataset**

In [ ]:
def cmerge_csv_files(cfolder_path):
    dfs = []
    error_files = []

    for filename in os.listdir(cfolder_path):
        if filename.endswith('.csv'):
            print('reading', filename)
            cfile_path = os.path.join(cfolder_path, filename)
            try:
                # Attempt to read the CSV file
                df = pd.read_csv(cfile_path)
                dfs.append(df)
            except pd.errors.ParserError as e:
                # If a ParserError occurs, print the file path and re-raise the error
                print(f"Error in file: {cfile_path}")
                ##proceed even if there's an error in some files
                #raise e
                error_files.append(filename)

    # Proceed to merge dataframes even if errors were raiserd
    cmerged_df = pd.concat(dfs, ignore_index=True)
    if len(error_files) > 0:
        print("The following csvs threw parsing errors and were not merged:", str(error_files))
    return cmerged_df

In [ ]:
df2 = cmerge_csv_files(cfolder_path)
print("len df2:", len(df2))
print("columns:", df2.columns)
df2.sample(5)

reading c_cases2015.csv
reading c_cases2016.csv
reading c_cases2017.csv
reading c_cases2018.csv
reading c_cases2019.csv
reading c_cases2020.csv
reading c_cases2021.csv
reading c_cases2022.csv
reading c_cases2023.csv
len df2: 148360
columns: Index(['Case Type', 'Region', 'Case Number', 'Case Name', 'Status',
       'Date Filed', 'Date Closed', 'Reason Closed', 'City',
       'States & Territories', 'Employees on charge/petition', 'Allegations',
       'Participants'],
      dtype='object')


,Case Type,Region,Case Number,Case Name,Status,Date Filed,Date Closed,Reason Closed,City,States & Territories,Employees on charge/petition,Allegations,Participants
28793,C,"Region 21, Los Angeles, California",21-CA-180664,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",Closed,07/21/2016,10/25/2016,Withdrawal Non-adjusted,Oceanside,CA,20.0,8(a)(1) Coercive Rules\n,"Charging Party, Legal Representative, WYNNE ED..."
110940,C,"Region 15, New Orleans, Louisiana",15-CA-287256,United States Postal Service,Closed,12/06/2021,12/14/2021,Withdrawal Non-adjusted,Panama City,FL,40.0,8(a)(5) Refusal to Bargain/Bad Faith Bargainin...,"Charged Party / Respondent, Legal Representati..."
26068,C,"Region 21, Los Angeles, California",21-CA-183792,UNITED STATES POSTAL SERVICE,Closed,09/06/2016,01/23/2017,Withdrawal Non-adjusted,WINTERHAVEN,CA,5.0,8(a)(3) Changes in Terms and Conditions of Emp...,"Charged Party / Respondent, Legal Representati..."
37377,C,"Region 06, Pittsburgh, Pennsylvania",06-CA-170776,"Johnny on the Spot, Ltd., Lighthouse Medical, ...",Closed,03/01/2016,04/27/2016,Withdrawal Adjusted,Altoona,PA,7.0,8(a)(5) Repudiation/Modification of Contract [...,"Charged Party / Respondent, Legal Representati..."
36370,C,"Region 19, Seattle, Washington",19-CA-172237,"Corliss Resources, Inc.",Closed,03/17/2016,07/13/2016,Dismissal Non-adjusted,Sumner,WA,35.0,"8(a)(1) Coercive Actions (Surveillance, etc)\n...","Charged Party / Respondent, Legal Representati..."


### 3c. **Load Related Case Dataset**

In [ ]:
#Make json files function (similar to df1 and df2 funcs)
def merge_relatedcase_json_files(path=jsonfolder_path):
  json_files = []

  for filename in os.listdir(path):
    if filename.startswith('related'):
      json_files.append(jsonfolder_path + "/" + filename)
  #print("json files to read:", json_files)

  all_cases = []
  #for file_path in
  return None

merge_relatedcase_json_files()

In [ ]:
json_files = []

for filename in os.listdir(jsonfolder_path):
    if filename.startswith('related'):
      json_files.append(jsonfolder_path + "/" + filename)

# Initialize an empty list to collect all case data
all_cases = []

# Iterate over each JSON file
for file_path in json_files:
    with open(file_path, 'r') as file:
        data = json.load(file)

        # Iterate over each case in the JSON file
        for case_number, case_details in data.items():
            if len(case_details['Related Cases']) > 0:
                # Add the case number to the case details dictionary
                case_details['Case Number'] = case_number

                # If there are related cases, handle them as needed
                # For simplicity, this example just counts the number of related cases
                case_details['Number of Related Cases'] = len(case_details.get('Related Cases', []))

                # Append the case details to the list
                all_cases.append(case_details)


In [ ]:
# Convert the list of all case details into a DataFrame #len df 3: 19922
df3 = pd.DataFrame(all_cases)
print("len df 3:", len(df3))

len df 3: 838


In [ ]:
df3.sample(3)

,Case Name,Case URL,Date Filed,Status,No. of Employees,Location,Region Assigned,Tally Information,Docket Activity,Related Documents,Allegations,Participants,Related Cases,Case Number,Number of Related Cases
202,Albertsons LLC,https://www.nlrb.gov/case/19-RC-285967,,,,,,[],[],[],[],[],"[{'Case Number': '19-RC-285965', 'Case URL': '...",19-RC-285967,2
330,"STAFFORD-SMITH, INC.",https://www.nlrb.gov/case/25-RC-228626,,,,,,[],[],[],[],[],"[{'Case Number': '25-CA-229649', 'Case URL': '...",25-RC-228626,3
7,Waxwork Records,https://www.nlrb.gov/case/15-RC-330459,,,,,,[],[],[],[],[],"[{'Case Number': '15-RM-330852', 'Case URL': '...",15-RC-330459,1


# **4. Run Initial Checks and Assertions**

In [ ]:
#Make sure that all R-cases, C-cases, and related cases are loaded properly
def print_dataframe_lengths(*dataframes):

    for idx, df in enumerate(dataframes, start=1):
        print(f"Length of df{idx}: {len(df)}")
print_dataframe_lengths(df, df2, df3)

Length of df1: 60075
Length of df2: 148360
Length of df3: 838


### Check R Case Dataset

In [ ]:
num_unique_case_numbers = df['Case Number'].nunique()
num_unique_case_numbers, df3['Case Number'].nunique()

(19851, 838)

In [ ]:
df.columns

Index(['Case Type', 'Region', 'Case Number', 'Case Name', 'Status',
       'Date Filed', 'Date Closed', 'Reason Closed', 'City',
       'States & Territories', 'Employees on charge/petition', 'Participants',
       'Union', 'Unit Sought', 'Voters'],
      dtype='object')

In [ ]:
# Double-checking the DataFrame right before conversion
print(df.columns)

# Convert 'Date Filed' to datetime and check the date range
df['Date Filed'] = pd.to_datetime(df['Date Filed'])
earliest_date = df['Date Filed'].min()
latest_date = df['Date Filed'].max()

print(f"Earliest date: {earliest_date}")
print(f"Latest date: {latest_date}")


Index(['Case Type', 'Region', 'Case Number', 'Case Name', 'Status',
       'Date Filed', 'Date Closed', 'Reason Closed', 'City',
       'States & Territories', 'Employees on charge/petition', 'Participants',
       'Union', 'Unit Sought', 'Voters'],
      dtype='object')
Earliest date: 2015-01-02 00:00:00
Latest date: 2023-12-28 00:00:00


### Check C Case Dataset

In [ ]:
df2['Date Filed'] = pd.to_datetime(df['Date Filed'])

# Find the earliest (min) and latest (max) dates in the 'Date Filed' column
earliest_date = df2['Date Filed'].min()
latest_date = df2['Date Filed'].max()

print(f"Earliest date: {earliest_date}")
print(f"Latest date: {latest_date}")

Earliest date: 2015-01-02 00:00:00
Latest date: 2023-12-28 00:00:00


In [ ]:
df2.sample(3)

,Case Type,Region,Case Number,Case Name,Status,Date Filed,Date Closed,Reason Closed,City,States & Territories,Employees on charge/petition,Allegations,Participants
116375,C,"Region 16, Fort Worth, Texas",16-CA-280497,United States Postal Service,Closed,NaT,07/30/2021,Withdrawal Adjusted,San Antonio,TX,75.0,8(a)(5) Refusal to Furnish Information\n,"Charged Party / Respondent, Legal Representati..."
118386,C,"Region 14, Saint Louis, Missouri",14-CA-277887,United States Postal Service,Closed,NaT,01/25/2022,Dismissal Adjusted,Tulsa,OK,500.0,"8(a)(1) Coercive Statements (Threats, Promises...","Charged Party / Respondent, Legal Representati..."
67714,C,"Region 07, Detroit, Michigan",07-CA-224453,Cambridge East Healthcare Center,Closed,NaT,08/06/2018,Withdrawal Adjusted,Madison Heights,MI,100.0,8(a)(3) Discharge (Including Layoff and Refusa...,"Charged Party / Respondent, Employer, Cambridg..."


### Check Related Case Dataset

In [ ]:
num_unique_case_numbers2 = df3['Case Number'].nunique()
num_unique_case_numbers2

838

# **5. Clean and Process Data**

#### Remove Duplicates

In [ ]:
def drop_duplicate_cases(dataframe):
    """
    Drops rows with duplicate values in the 'Case Number' column for a given DataFrame in place.

    """
    # Drop duplicate rows based on the 'Case Number' column in place
    dataframe.drop_duplicates(subset=['Case Number'], keep='first', inplace=True)

In [ ]:
drop_duplicate_cases(df)
drop_duplicate_cases(df2)
drop_duplicate_cases(df3)

In [ ]:
def print_dataframe_lengths(*dataframes):

    for idx, df in enumerate(dataframes, start=1):
        print(f"Length of df{idx}: {len(df)}")
print_dataframe_lengths(df, df2, df3)

Length of df1: 19851
Length of df2: 145712
Length of df3: 838


#### Check Nans and Replace Missing Values

In [ ]:
# Function to check for NaNs in all columns
def check_nans(df):
    nans = df.isna().sum()  # Get the count of NaNs in each column
    nans = nans[nans > 0]  # Filter out columns with no NaNs
    return nans

In [ ]:
# Using the function and printing the result
nan_info = check_nans(df2)
print("Columns with NaNs and their counts:")
print(nan_info)

# Assert statement to test if the function works correctly
#assert nan_info.equals(df.isna().sum()[df.isna().sum() > 0]), "The NaN check does not match expected output."


Columns with NaNs and their counts:
Date Filed                      86513
Employees on charge/petition      598
Allegations                       175
Participants                       14
dtype: int64


In [ ]:
def convert_missing_to_nan(*dataframes):
    """
    Convert missing values in numerical columns of each DataFrame provided as input to NaN.
    """
    for df in dataframes:
        # Iterate over columns and convert missing values to NaN
        for col in df.select_dtypes(include=['number']).columns:
            df[col] = df[col].replace('', float('nan'))

In [ ]:
convert_missing_to_nan(df, df2, df3)

#### Convert Column Names and String Columns to Lowercase

In [ ]:
#This is to normalize the column names to make them easier to manipulate later on in the pipeline.
def to_lowercase(value):
    if isinstance(value, str):
        return value.lower()
    elif isinstance(value, list):
        return [to_lowercase(item) for item in value]
    elif isinstance(value, dict):
        return {k.lower(): to_lowercase(v) for k, v in value.items()}
    return value

def convert_df_to_lowercase(dataframe):
    dataframe.columns = [col.lower() for col in dataframe.columns]
    for column in dataframe.columns:
        dataframe[column] = dataframe[column].apply(to_lowercase)
    return dataframe

In [ ]:
df = convert_df_to_lowercase(df)
df2 = convert_df_to_lowercase(df2)
df3 = convert_df_to_lowercase(df3)

In [ ]:
def print_dataframe_lengths(*dataframes):

    for idx, df in enumerate(dataframes, start=1):
        print(f"Length of df{idx}: {len(df)}")

In [ ]:
type(df3['related cases'][0][0]) #still a list of dicts

dict

In [ ]:
print_dataframe_lengths(df, df2, df3)

Length of df1: 19851
Length of df2: 145712
Length of df3: 838


### R Case Dataset

#### Drop Unnecessary Columns

In [ ]:
columns_to_drop = ['case type','city', 'states & territories','union', 'unit sought','region', 'status','participants',]

df = df.drop(columns=columns_to_drop)

In [ ]:
def list_data_types(df):

    data_types = {}
    for column in df.columns:
        data_types[column] = df[column].dtype
    return data_types

# List data types for each column
column_data_types = list_data_types(df)
print("Data types for each column:")
column_data_types

Data types for each column:


{'case number': dtype('O'),
 'case name': dtype('O'),
 'date filed': dtype('<M8[ns]'),
 'date closed': dtype('O'),
 'reason closed': dtype('O'),
 'employees on charge/petition': dtype('float64'),
 'voters': dtype('float64')}

In [ ]:
#Ensure that all remaining columns are of types that pandas understands.
columns_to_convert = ['case number', 'case name', 'reason closed', ]

for column in columns_to_convert:
    df[column] = df[column].astype(str)

df['date filed'] = pd.to_datetime(df['date filed'])
df['date closed'] = pd.to_datetime(df['date closed'])

df['employees on charge/petition'] = df['employees on charge/petition'].astype('Int64')
df['voters'] = df['voters'].astype('Int64')

In [ ]:
def strip_hyphens_from_case_number(df):

    # Check if the 'case number' column exists
    if 'case number' not in df.columns:
        raise ValueError("The DataFrame does not have a 'case number' column.")

    # Strip hyphens from the 'case number' column
    df['case number'] = df['case number'].str.replace("-", "", regex=False)

    return df

In [ ]:
df = strip_hyphens_from_case_number(df)

#### Remove Non Relevant Case Types

In [ ]:
# List of identifiers to remove - these cases were not applicable to the research questions that we had posed.
identifiers = ['uc', 'rd', 'rm', 'ud', 'ac']

# Regular expression that matches any of the identifiers
regex_pattern = '|'.join(identifiers)

# Drop rows where 'case number' contains any of the identifiers
df = df[~df['case number'].str.contains(regex_pattern, na=False)]


# Assert statement to ensure no rows with these identifiers exist
assert not df['case number'].str.contains(regex_pattern).any(), "Some rows with specified identifiers still exist."

In [ ]:
df.sample(5)

,case number,case name,date filed,date closed,reason closed,employees on charge/petition,voters
13188,32rc205743,"sutter shared lab, llc",2017-09-07,2017-10-24,certification of results,71,70
6304,21rc275324,"centerline logistics corporation, olympic tug ...",2021-04-08,2022-09-28,certific. of representative,25,45
14932,14rc190018,st. alexius hospital corporation 1 d/b/a st. a...,2016-12-19,2017-01-23,certification of results,22,20
6625,04rc272006,21st century cyber charter school,2021-01-29,2021-09-08,certific. of representative,93,88
14573,05rc193359,command security corporation,2017-02-17,2017-03-23,certific. of representative,50,54


In [ ]:
def print_dataframe_lengths(*dataframes_with_names):
    for name, df in dataframes_with_names:
        print(f"{name}: {len(df)}")

print_dataframe_lengths(('df', df), ('df2', df2), ('df3', df3))

df: 15885
df2: 145712
df3: 838


### C Case Dataset

In [ ]:
columns_to_drop2 = ['case type','city', 'states & territories','region', 'status','participants']


df2 = df2.drop(columns=columns_to_drop2)

In [ ]:
def list_data_types(df):

    data_types = {}
    for column in df.columns:
        data_types[column] = df[column].dtype
    return data_types


# List data types for each column
column_data_types2 = list_data_types(df2)
print("Data types for each column:")
column_data_types2

Data types for each column:


{'case number': dtype('O'),
 'case name': dtype('O'),
 'date filed': dtype('<M8[ns]'),
 'date closed': dtype('O'),
 'reason closed': dtype('O'),
 'employees on charge/petition': dtype('float64'),
 'allegations': dtype('O')}

In [ ]:
columns_to_convert = ['case number', 'case name', 'reason closed', 'allegations']
for column in columns_to_convert:
    df2[column] = df2[column].astype(str)

df2['date filed'] = pd.to_datetime(df['date filed'])
df2['date closed'] = pd.to_datetime(df['date closed'])

df2['employees on charge/petition'] = df['employees on charge/petition'].astype('Int64')

In [ ]:
def strip_hyphens_from_case_number(df):
    # Check if the 'case number' column exists
    if 'case number' not in df.columns:
        raise ValueError("The DataFrame does not have a 'case number' column.")

    # Strip hyphens from the 'case number' column
    df['case number'] = df['case number'].str.replace("-", "", regex=False)

    return df

In [ ]:
df2 = strip_hyphens_from_case_number(df2)

#### Remove Non Relevant Case Types

In [ ]:
# List of identifiers to remove
identifiers = ['cp', 'ce']

# Regular expression that matches any of the identifiers
regex_pattern = '|'.join(identifiers)

# Drop rows where 'case number' contains any of the identifiers
df2 = df2[~df2['case number'].str.contains(regex_pattern, na=False)]

# Assert statement to ensure no rows with these identifiers exist
assert not df2['case number'].str.contains(regex_pattern).any(), "Some rows with specified identifiers still exist."

In [ ]:
def print_dataframe_lengths(*dataframes_with_names):
    for name, df in dataframes_with_names:
        print(f"{name}: {len(df)}")

print_dataframe_lengths(('df', df), ('df2', df2), ('df3', df3))

df: 15885
df2: 145536
df3: 838


### Related Cases Dataset

In [ ]:
def list_data_types(df):

    data_types = {}
    for column in df.columns:
        data_types[column] = df[column].dtype
    return data_types


# List data types for each column
column_data_types3 = list_data_types(df3)
print("Data types for each column:")
column_data_types3

Data types for each column:


{'case name': dtype('O'),
 'case url': dtype('O'),
 'date filed': dtype('O'),
 'status': dtype('O'),
 'no. of employees': dtype('O'),
 'location': dtype('O'),
 'region assigned': dtype('O'),
 'tally information': dtype('O'),
 'docket activity': dtype('O'),
 'related documents': dtype('O'),
 'allegations': dtype('O'),
 'participants': dtype('O'),
 'related cases': dtype('O'),
 'case number': dtype('O'),
 'number of related cases': dtype('int64')}

In [ ]:
# List of columns to be dropped from df3
columns_to_drop = [ 'date filed', 'status', 'case name', 'case url','no. of employees', 'location',
                    'region assigned', 'tally information', 'docket activity', 'related documents',
                    'allegations', 'participants' ]

# Drop the specified columns from df3, if they exist
df3 = df3.drop(columns=[col for col in columns_to_drop if col in df3.columns], errors='ignore')

In [ ]:
columns_to_convert = ['case number']

for column in columns_to_convert:
    df3[column] = df3[column].astype(str)

df3['number of related cases'] = df3['number of related cases'].astype('Int64')

In [ ]:
def strip_hyphens_from_case_number(df):
    # Check if the 'case number' column exists
    if 'case number' not in df.columns:
        raise ValueError("The DataFrame does not have a 'case number' column.")

    # Strip hyphens from the 'case number' column
    df['case number'] = df['case number'].str.replace("-", "", regex=False)

    return df

In [ ]:
df3 = strip_hyphens_from_case_number(df3)

In [ ]:
df3.sample(5)
#There is a problem with df3. Or maybe there's not a problem but I just need to get rid of all the extra columns, idk.

,related cases,case number,number of related cases
388,"[{'case number': '18-ca-222554', 'case url': '...",18rd220711,1
224,"[{'case number': '22-rc-279583', 'case url': '...",22rc279729,1
528,"[{'case number': '21-ca-191204', 'case url': '...",21rc190832,1
614,"[{'case number': '21-ca-149383', 'case url': '...",21rd160271,5
329,"[{'case number': '12-ac-255860', 'case url': '...",12rc228892,1


#### Remove Non Relevant Cases

> Indented block



In [ ]:
# List of identifiers to remove
identifiers = ['uc', 'rd', 'rm', 'ud', 'ac']

# Regular expression that matches any of the identifiers
regex_pattern = '|'.join(identifiers)

# Drop rows where 'case number' contains any of the identifiers
df3 = df3[~df3['case number'].str.contains(regex_pattern, na=False)]


# Assert statement to ensure no rows with these identifiers exist
assert not df3['case number'].str.contains(regex_pattern).any(), "Some rows with specified identifiers still exist."

In [ ]:
df3.sample(5)

,related cases,case number,number of related cases
307,"[{'case number': '21-rc-238208', 'case url': '...",21rc238430,3
775,"[{'case number': '05-ca-177884', 'case url': '...",05rc177566,3
321,"[{'case number': '08-ca-233009', 'case url': '...",08rc232661,1
207,"[{'case number': '19-rc-283239', 'case url': '...",19rc284052,4
418,"[{'case number': '04-rc-211814', 'case url': '...",04rc211792,1


In [ ]:
def list_data_types(df):

    data_types = {}
    for column in df.columns:
        data_types[column] = df[column].dtype
    return data_types


# List data types for each column
column_data_types3 = list_data_types(df3)
print("Data types for each column:")
column_data_types3

Data types for each column:


{'related cases': dtype('O'),
 'case number': dtype('O'),
 'number of related cases': Int64Dtype()}

In [ ]:
len(df3)

547

In [ ]:
def print_dataframe_lengths(*dataframes_with_names):
    for name, df in dataframes_with_names:
        print(f"{name}: {len(df)}")

print_dataframe_lengths(('df', df), ('df2', df2), ('df3', df3))
df2.sample(5)

df: 15885
df2: 145536
df3: 547


,case number,case name,date filed,date closed,reason closed,employees on charge/petition,allegations
143806,16ca318097,odfjell terminals houston,NaT,NaT,withdrawal non-adjusted,<NA>,8(a)(5) repudiation/modification of contract [...
83988,10ca247211,primary care of southwest georgia,NaT,NaT,withdrawal non-adjusted,<NA>,"8(a)(1) concerted activities (retaliation, dis..."
132259,18ca297521,minnesota public radio/apmg,NaT,NaT,withdrawal non-adjusted,<NA>,8(a)(5) refusal to bargain/bad faith bargainin...
51646,13ca199912,"tri city foods, inc. d/b/a burger king",NaT,NaT,informal settlement,<NA>,8(a)(3) changes in terms and conditions of emp...
77633,22cd213030,northeast regional council of carpenters (regi...,NaT,NaT,withdrawal non-adjusted,<NA>,8(b)(4)(d) all allegations\n


### Export cleaned data to cleaned_merged_data for further analysis. Will overwrite.

In [ ]:
# Overwrite data? [df, df2, df3]
overwrite_data = [False, False, False] #[True, True, True]

if overwrite_data[0] == True:
  df.to_csv((cleaned_data_path + "/df1_R_cases.csv"), index=False)
if overwrite_data[1] == True:
  df2.to_csv((cleaned_data_path + "/df2_C_cases.csv"), index=False)
if overwrite_data[2] == True:
  df3.to_csv((cleaned_data_path + "/df3_json_data.csv"), index=False)

# **6. Merge Datasets**

In [ ]:
dftest = df.copy()
df2test = df2.copy()
# Set 'case number' column as the index of the DataFrame
df2test.set_index('case number', inplace=True)
df3test = df3.copy()

In [ ]:
def print_dataframe_lengths(*dataframes_with_names):
    for name, df in dataframes_with_names:
        print(f"{name}: {len(df)}")

print_dataframe_lengths(('dftest', dftest), ('df2test', df2test), ('df3test', df3test))

dftest: 15885
df2test: 145536
df3test: 547


### Join R Cases and Related Cases on Case Number

In [ ]:
r_merge = pd.merge(dftest, df3test, on='case number', how='left')

In [ ]:
r_merge['related cases'] = r_merge['related cases'].fillna({})
r_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15885 entries, 0 to 15884
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   case number                   15885 non-null  object        
 1   case name                     15885 non-null  object        
 2   date filed                    15885 non-null  datetime64[ns]
 3   date closed                   15885 non-null  datetime64[ns]
 4   reason closed                 15885 non-null  object        
 5   employees on charge/petition  15859 non-null  Int64         
 6   voters                        11379 non-null  Int64         
 7   related cases                 544 non-null    object        
 8   number of related cases       544 non-null    Int64         
dtypes: Int64(3), datetime64[ns](2), object(4)
memory usage: 1.3+ MB


In [ ]:
r_merge.sample(5)

,case number,case name,date filed,date closed,reason closed,employees on charge/petition,voters,related cases,number of related cases
4801,03rc278264,"ethosenergy power plant services, llc",2021-06-09,2021-07-16,certific. of representative,18,17,NaN,<NA>
13116,21rc174278,american red cross,2016-04-18,2016-05-10,certific. of representative,6,6,NaN,<NA>
8289,31rc235164,dignity community care d/b/a northridge hospit...,2019-02-01,2019-03-05,certific. of representative,8,8,NaN,<NA>
9628,08rc216433,"amerigas propane, inc.",2018-03-13,2018-04-19,certification of results,10,10,NaN,<NA>
1742,13rc311833,"reliant transportation, inc.",2023-02-09,2023-03-22,certific. of representative,2,2,NaN,<NA>


In [ ]:
def list_data_types(df):

    data_types = {}
    for column in df.columns:
        data_types[column] = df[column].dtype
    return data_types


# List data types for each column
column_data_types4 = list_data_types(r_merge)
print("Data types for each column:")
column_data_types4

Data types for each column:


{'case number': dtype('O'),
 'case name': dtype('O'),
 'date filed': dtype('<M8[ns]'),
 'date closed': dtype('<M8[ns]'),
 'reason closed': dtype('O'),
 'employees on charge/petition': Int64Dtype(),
 'voters': Int64Dtype(),
 'related cases': dtype('O'),
 'number of related cases': Int64Dtype()}

##### convert related cases back to dictionary and clean case names.

In [ ]:
def clean_related_cases_data(df):
    for index, row in df.iterrows():
        if type(row['related cases']) is float:
            df.at[index, 'related cases'] = []  # Assign {} to the DataFrame
        elif len(row['related cases']) > 0:
            for item in row['related cases']:
                item['case number'] = item['case number'].replace("-", "")
    return df

r_merge = clean_related_cases_data(r_merge)

In [ ]:
def print_dataframe_lengths(*dataframes_with_names):
    for name, df in dataframes_with_names:
        print(f"{name}: {len(df)}")

print_dataframe_lengths(('r_merge', r_merge), ('df2test', df2test))

r_merge: 15885
df2test: 145536


#### Extract Relevant Case Numbers From DF2 Using Relevant Cases As The Guide

In [ ]:
all_related_case_numbers = set()
for value in r_merge['related cases']:
  if len(value) > 0:
    for item in value:
      all_related_case_numbers.update({item['case number']})
print(len(all_related_case_numbers))

871


In [ ]:
def add_index_as_column(df, column_name='case number'):
    # Copy the index of the DataFrame to a new column and convert it to string format
    df[column_name] = df.index.astype(str)
    return df

def filter_df_based_on_unique_values(df1=r_merge, df2=df2test):
    # Add 'case_number' column from the index
    df2 = add_index_as_column(df2)

    # Initialize an empty set to store all unique 'rel case num X' values
    all_related_case_numbers = set()

    # Iterate over each specified column and add unique values to the set
    for value in df1['related cases']:
      if len(value) > 0:
        for item in value:
          all_related_case_numbers.update({item['case number']})

    print("Found", len(all_related_case_numbers), "related case numbers.")
    # Filter df2test to keep only rows where the 'case_number' column matches any value in all_related_case_numbers
    df2_filtered = df2test[df2test['case number'].isin(all_related_case_numbers)]

    return df2_filtered

In [ ]:
df2_filtered = filter_df_based_on_unique_values(r_merge, df2test)
print(len(df2_filtered))


Found 871 related case numbers.
441


In [ ]:
df2_cases = set((df2_filtered['case number'].values))
print(len(df2_cases), type(df2_cases))

441 <class 'set'>


In [ ]:
rel_case_nums = []
for index, row in r_merge.iterrows():
  if len(row['related cases']) > 0:
    for x in row['related cases']:
     rel_case_nums.append((x['case number']))
rel_case_nums = set(rel_case_nums)
print(len(rel_case_nums), type(rel_case_nums))

871 <class 'set'>


In [ ]:
def add_rel_case_info(df1=r_merge, df2=df2_filtered):
  data_to_add = []
  for index, row in df1.iterrows():
    rowdict = {
    'rel_case_nums': [],
    'rel_dates_filed': [],
    'rel_dates_closed': [],
    'rel_reasons_closed': [],
    'rel_allegations': []
    }
    if len(row['related cases']) > 1:
      for item in row['related cases']:
        if item['case number'] in df2['case number']:
          caseinfo = df2.loc[item['case number']]
          rowdict['rel_case_nums'].append(caseinfo['case number'])
          rowdict['rel_dates_filed'].append(caseinfo['date filed'])
          rowdict['rel_dates_closed'].append(caseinfo['date closed'])
          rowdict['rel_reasons_closed'].append(caseinfo['reason closed'])
          rowdict['rel_allegations'].append(caseinfo['allegations'])
        else: pass
    data_to_add.append(rowdict)
  df_to_add = pd.DataFrame(data_to_add)
  merged_df = pd.concat([df1, df_to_add], axis=1)
  return merged_df



#### Match The C Cases To The Corresponding R Cases

In [ ]:
#The big pig!!
merged_data = add_rel_case_info(r_merge, df2_filtered)

In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15885 entries, 0 to 15884
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   case number                   15885 non-null  object        
 1   case name                     15885 non-null  object        
 2   date filed                    15885 non-null  datetime64[ns]
 3   date closed                   15885 non-null  datetime64[ns]
 4   reason closed                 15885 non-null  object        
 5   employees on charge/petition  15859 non-null  Int64         
 6   voters                        11379 non-null  Int64         
 7   related cases                 15885 non-null  object        
 8   number of related cases       544 non-null    Int64         
 9   rel_case_nums                 15885 non-null  object        
 10  rel_dates_filed               15885 non-null  object        
 11  rel_dates_closed            

In [ ]:
merged_data.sample(5)

,case number,case name,date filed,date closed,reason closed,employees on charge/petition,voters,related cases,number of related cases,rel_case_nums,rel_dates_filed,rel_dates_closed,rel_reasons_closed,rel_allegations
13895,29rc164811,edison home health care - kings adult care center,2015-11-23,2015-12-01,withdrawal non-adjusted,35,<NA>,"[{'case number': '29rc164800', 'case url': 'ht...",2,[],[],[],[],[]
3017,14rc296784,starbucks corporation,2022-05-31,2022-08-26,certific. of representative,33,33,[],<NA>,[],[],[],[],[]
2796,19rc299157,"cadman materials, inc.",2022-07-11,2022-09-19,certific. of representative,16,18,[],<NA>,[],[],[],[],[]
14302,04rc159980,the american bottling company,2015-09-15,2015-10-20,certification of results,30,29,[],<NA>,[],[],[],[],[]
11817,28rc190782,"transdev services, inc.",2017-01-04,2017-01-12,withdrawal non-adjusted,26,<NA>,[],<NA>,[],[],[],[],[]


In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15885 entries, 0 to 15884
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   case number                   15885 non-null  object        
 1   case name                     15885 non-null  object        
 2   date filed                    15885 non-null  datetime64[ns]
 3   date closed                   15885 non-null  datetime64[ns]
 4   reason closed                 15885 non-null  object        
 5   employees on charge/petition  15859 non-null  Int64         
 6   voters                        11379 non-null  Int64         
 7   related cases                 15885 non-null  object        
 8   number of related cases       544 non-null    Int64         
 9   rel_case_nums                 15885 non-null  object        
 10  rel_dates_filed               15885 non-null  object        
 11  rel_dates_closed            

#### Split Allegations Column, Categorize and Aggregate

In [ ]:
##won't work in this notebook, data reformatted
def split_and_categorize_allegations(df):
    # Remove '(' and ')' from allegations columns
    for i in range(1, 11):  # For 'r1_allegations' through 'r10_allegations'
        column_name = f'r{i}_allegations'
        if column_name in df.columns:
            df[column_name] = df[column_name].str.replace(r'\(|\)', '', regex=True)

    # Updated charge types
    charge_types = ['8a1', '8a2', '8a3', '8a4', '8a5']

    # Initialize columns for charge presence (binary) and counts (aggregate)
    for charge in charge_types:
        df[f'{charge}_binary'] = 0
        df[f'{charge}_count'] = 0

    # Loop through each allegations column and process charge types
    for i in range(1, 11):
        column_name = f'r{i}_allegations'
        # Check if the column exists to handle cases where some columns might not be present
        if column_name in df.columns:
            for charge in charge_types:
                # Check for presence of charge type in the column and update binary column
                df[f'{charge}_binary'] |= df[column_name].str.contains(charge, na=False).astype(int)

                # Count instances and update aggregate column
                df[f'{charge}_count'] += df[column_name].str.contains(charge, na=False).astype(int)

    return df


In [ ]:
## Export merged data. Will overwrite!!!
export = False
if export == True:
  merged_data.to_csv((cleaned_data_path + "/merged_data.csv"), index=False)

In [ ]:
merged_data2 = split_and_categorize_allegations(merged_data)

In [ ]:
merged_data2.sample(10)

,case number,case name,date filed,date closed,reason closed,employees on charge/petition,voters,related cases,number of related cases,rel_case_nums,...,8a1_binary,8a1_count,8a2_binary,8a2_count,8a3_binary,8a3_count,8a4_binary,8a4_count,8a5_binary,8a5_count
5137,20rc273993,dignity health medical foundation - north sta...,2021-03-11,2021-06-02,certification of results,21,21,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
9197,14rc221943,"laura ford of sullivan, llc; laura chevrolet b...",2018-06-13,2018-07-06,withdrawal non-adjusted,14,<NA>,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
10576,15rc204599,delek us,2017-08-18,2017-08-22,withdrawal non-adjusted,4,<NA>,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
70,05rc331234,walden security,2023-12-04,2023-12-14,withdrawal non-adjusted,15,<NA>,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
5712,20rc265928,"n leasing company, llc",2020-09-11,2020-12-03,certification of results,39,37,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
9685,05rc215802,omniplex world services corporation,2018-03-02,2018-04-30,certific. of representative,139,157,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
6808,12rc252190,"healthcare services group, inc.",2019-11-21,2019-12-23,certific. of representative,22,21,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
4442,12rc283049,"intermodal mexico usa, llc",2021-09-16,2021-12-10,certific. of representative,42,50,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
3145,20rc295447,"matson terminals, inc.",2022-05-09,2022-07-21,certific. of representative,2,2,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0
3736,01rc290605,"charter communications, inc. d/b/a spectrum",2022-02-15,2023-03-14,certific. of representative,30,26,[],<NA>,[],...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def print_dataframe_lengths(*dataframes):

    for idx, df in enumerate(dataframes, start=1):
        print(f"Length of df{idx}test: {len(df)}")
print_dataframe_lengths(dftest, df2test, df3test)

Length of df1test: 15885
Length of df2test: 145536
Length of df3test: 547


## **Statistics Models**

#### Potential Aggregation and Normalization Functions

In [ ]:
# Function to aggregate ulp data by year and normalize it

def aggregate_and_normalize(data_processed, binary_columns):


    # Count the total number of ulps per year
    total_ulps_per_year = data_processed.drop_duplicates(subset=['xxx']).groupby('xxx').size()

    # Aggregate the sum of binary indicators for each year
    yearly_data = data_processed.groupby('xxx')[binary_columns].sum()

    # Initialize an empty DataFrame to hold the filtered yearly data
    filtered_yearly_data = pd.DataFrame(index=yearly_data.index)

    # Filter out years with no incidents for each binary indicator
    for col in binary_columns:
        # Find the first year with a non-zero count for the indicator
        first_valid_year = yearly_data[col].ne(0).idxmax()
        # Keep data from the first valid year to the present for the indicator
        filtered_yearly_data[col] = yearly_data[col].loc[first_valid_year:]

    # Add the total number of incidents to the filtered data
    filtered_yearly_data['Total Incidents'] = total_incidents_per_year.loc[filtered_yearly_data.index]

    # Create a scaler object to normalize the data
    scaler = MinMaxScaler()
    # Normalize the data and create a new DataFrame with the normalized values
    yearly_data_normalized = pd.DataFrame(scaler.fit_transform(filtered_yearly_data), columns=filtered_yearly_data.columns, index=filtered_yearly_data.index)

    # Calculate a 3-year moving average for the normalized data
    yearly_data_ma = yearly_data_normalized.rolling(window=3).mean()

    # Return the DataFrame with the normalized and smoothed data
    return yearly_data_ma


#### Potential Stats Parameters

In [ ]:
def calculate_stats_params(data_processed, binary_columns):

    # Initialize a dictionary to store the statistical parameters for each column
    stats_params = {}

    # Aggregate the data by 'Year' and sum the binary columns to get yearly sums
    yearly_aggregated_data = data_processed.groupby('Year')[binary_columns].sum()

    # Calculate the mean and standard deviation of these yearly sums for each binary column
    for column in binary_columns:
        # Calculate the mean for the current column over all years
        yearly_mean = yearly_aggregated_data[column].mean()
        # Calculate the standard deviation for the current column over all years
        yearly_std_dev = yearly_aggregated_data[column].std()
        # Store the calculated mean and standard deviation in the stats_params dictionary
        stats_params[column] = {
            'yearly_mean': yearly_mean,
            'yearly_std_dev': yearly_std_dev
        }

    # Calculate statistical parameters for the total number of incidents per year
    # Remove duplicate incidents based on 'Incident ID' to ensure unique counts
    total_incidents = data_processed.drop_duplicates(subset=['Incident ID']).groupby('Year').size()
    # Calculate and store the mean and standard deviation for the total incidents
    stats_params['Total Incidents'] = {
        'yearly_mean': total_incidents.mean(),
        'yearly_std_dev': total_incidents.std()
    }

    # Return the dictionary containing the statistical parameters for each column
    return stats_params
